In [1]:
!pip install python-dotenv


^C


In [1]:
pip install pyodbc


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyodbc
import os
from dotenv import load_dotenv
import hashlib

load_dotenv()

# connection to MSSQL server
def create_db_connection():
    conn = pyodbc.connect(
        'DRIVER={SQL Server};'
        f'SERVER={os.getenv("SQL_SERVER")};'
        f'DATABASE={os.getenv("SQL_DATABASE")};'  
        'Trusted_Connection=yes;'
    )
    return conn

# Hash the password for security
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

#  if the username already exists in the database
def check_username_exists(username):
    conn = create_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM dbo.Users WHERE username=?", (username,))  # Using dbo.Users
    user = cursor.fetchone()
    conn.close()
    return user is not None

 #Sign up function
def signup():
    username = input("Enter a new username: ")
    
    if check_username_exists(username):
        print("Username already exists. Please try again.")
        return
    
    password = input("Enter a password: ")
    hashed_password = hash_password(password)
    
    try:
        conn = create_db_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO dbo.Users (username, password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        conn.close()
        print(f"User {username} created successfully!")
    except Exception as e:
        print("Error during sign up:", e)

#Login function
def login():
    username = input("Enter username: ")
    password = input("Enter password: ")
    
    conn = create_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM dbo.Users WHERE username=?", (username,)) 
    user = cursor.fetchone()
    conn.close()

    if user:
        stored_password = user[2]  # Retrieve the stored hashed password (index 2 for password column)
        if stored_password == hash_password(password):
            print(f"Welcome back, {username}!")
        else:
            print("Incorrect password. Please try again.")
    else:
        print("Username not found. Please try again.")

 #Main function for handling user interaction
def main():
    while True:
        print("\n1. Sign Up")
        print("2. Log In")
        print("3. Exit")
        choice = input("Select an option (1/2/3): ")
        
        if choice == '1':
            signup()
        elif choice == '2':
            login()
        elif choice == '3':
            print("Goodbye!,Thank you")
            break
        else:
            print("Invalid option. Please try again.")

if __name__ == '__main__':
    main()



1. Sign Up
2. Log In
3. Exit
Select an option (1/2/3): 1
Enter a new username: kiran
Enter a password: kiran@123
User kiran created successfully!

1. Sign Up
2. Log In
3. Exit
Select an option (1/2/3): 2
Enter username: kiran
Enter password: kiran@123
Welcome back, kiran!

1. Sign Up
2. Log In
3. Exit
